In [2]:
import numpy as np
import pandas as pd

print('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [9]:
# The code was removed by Watson Studio for sharing.

count     103
unique    103
top       M6L
freq        1
Name: Postal Code, dtype: object

In [17]:
list_of_dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M') #read all tables
df = list_of_dfs[0] # first table is the one we need

# remove Not assigned Borough
df = df[df.Borough != 'Not assigned']

#df['Postal Code'].describe()
# Note : Table no longer contains duplicate

df[df.Neighbourhood == 'Not assigned']
# Note : Table no longer contains 'Not assigned' Neighbourhoods


,Postal Code,Borough,Neighbourhood
